In [1]:
# Import necessary libraries
import pandas as pd
import time
import math

In [2]:
# Load data
t1 = time.time()
users = pd.read_csv('Data/users.csv')
weights = pd.read_csv('Data/weights.csv')
treatments = pd.read_csv('Data/treatments.csv')
t2 = time.time()
print('Time taken to load datasets ', t2-t1)

Time taken to load datasets  0.06323099136352539


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   UID          1000 non-null   object 
 1   Name         1000 non-null   object 
 2   LastName     1000 non-null   object 
 3   Gender       1000 non-null   object 
 4   Unit         1000 non-null   int64  
 5   Birthday     1000 non-null   object 
 6   Age          1000 non-null   int64  
 7   Height       1000 non-null   int64  
 8   CreatedDate  1000 non-null   object 
 9   IsActive     1000 non-null   bool   
 10  ClinicID     1000 non-null   int64  
 11  loginId      0 non-null      float64
 12  success      1000 non-null   bool   
dtypes: bool(2), float64(1), int64(4), object(6)
memory usage: 88.0+ KB


In [4]:
weights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32073 entries, 0 to 32072
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MasterUserID  32073 non-null  object 
 1   Weight        32073 non-null  float64
 2   BMI           32073 non-null  float64
 3   BodyFat       32073 non-null  float64
 4   BodyWater     32073 non-null  float64
 5   Bone          32073 non-null  float64
 6   VisceralFat   32073 non-null  float64
 7   BMR           32073 non-null  float64
 8   MuscleMass    32073 non-null  float64
 9   CreatedDate   32073 non-null  object 
 10  UpdatedDate   32073 non-null  object 
 11  IsActive      32073 non-null  bool   
 12  IsDelete      32073 non-null  bool   
dtypes: bool(2), float64(8), object(3)
memory usage: 2.8+ MB


In [5]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   MasterUserID     1000 non-null   object
 1   TreatmentTypeID  1000 non-null   int64 
 2   StartDate        1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [6]:
# Left join dataframes: Users <> Weights, Treatments
t1= time.time()

uw = pd.merge(users, weights,
              left_on='UID', right_on='MasterUserID',
              how='left')
merged_df = pd.merge(uw, treatments,
                     left_on='UID', right_on='MasterUserID',
                     how='left')

t2 = time.time()
print('Time taken to merge datasets ', t2-t1)

Time taken to merge datasets  0.032557010650634766


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32073 entries, 0 to 32072
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   UID              32073 non-null  object 
 1   Name             32073 non-null  object 
 2   LastName         32073 non-null  object 
 3   Gender           32073 non-null  object 
 4   Unit             32073 non-null  int64  
 5   Birthday         32073 non-null  object 
 6   Age              32073 non-null  int64  
 7   Height           32073 non-null  int64  
 8   CreatedDate_x    32073 non-null  object 
 9   IsActive_x       32073 non-null  bool   
 10  ClinicID         32073 non-null  int64  
 11  loginId          0 non-null      float64
 12  success          32073 non-null  bool   
 13  MasterUserID_x   32073 non-null  object 
 14  Weight           32073 non-null  float64
 15  BMI              32073 non-null  float64
 16  BodyFat          32073 non-null  float64
 17  BodyWater   

In [8]:
# Check if MastersUserID column from `treatments` and `weights` are duplicates of `users` UID
print(sum(merged_df['UID']==merged_df['MasterUserID_x']))
print(sum(merged_df['UID']==merged_df['MasterUserID_y']))

# Yes, all the values match UID column

32073
32073


In [9]:
# Drop 2 duplicate UserID columns
t1 = time.time()

merged_df = merged_df.drop(columns=['MasterUserID_x', 'MasterUserID_y'])


t2 = time.time()
print('Time taken to drop columns ', t2-t1)
merged_df.info()

Time taken to drop columns  0.01725482940673828
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32073 entries, 0 to 32072
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   UID              32073 non-null  object 
 1   Name             32073 non-null  object 
 2   LastName         32073 non-null  object 
 3   Gender           32073 non-null  object 
 4   Unit             32073 non-null  int64  
 5   Birthday         32073 non-null  object 
 6   Age              32073 non-null  int64  
 7   Height           32073 non-null  int64  
 8   CreatedDate_x    32073 non-null  object 
 9   IsActive_x       32073 non-null  bool   
 10  ClinicID         32073 non-null  int64  
 11  loginId          0 non-null      float64
 12  success          32073 non-null  bool   
 13  Weight           32073 non-null  float64
 14  BMI              32073 non-null  float64
 15  BodyFat          32073 non-null  float64
 16  BodyWater 

In [10]:
# Rename columns for better understanding
columns_to_rename = {'CreatedDate_x': 'UIDCreatedDate',
                     'IsActive_x': 'User_IsActive',
                     'CreatedDate_y': 'Weights_CreatedDate',
                     'UpdatedDate': 'Weights_UpdatedDate',
                     'IsActive_y': 'Weights_IsActive',
                     'IsDelete': 'Weights_IsDelete',
                     'StartDate': 'Treatment_StartDate',
                     }
merged_df = merged_df.rename(columns=columns_to_rename)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32073 entries, 0 to 32072
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UID                  32073 non-null  object 
 1   Name                 32073 non-null  object 
 2   LastName             32073 non-null  object 
 3   Gender               32073 non-null  object 
 4   Unit                 32073 non-null  int64  
 5   Birthday             32073 non-null  object 
 6   Age                  32073 non-null  int64  
 7   Height               32073 non-null  int64  
 8   UIDCreatedDate       32073 non-null  object 
 9   User_IsActive        32073 non-null  bool   
 10  ClinicID             32073 non-null  int64  
 11  loginId              0 non-null      float64
 12  success              32073 non-null  bool   
 13  Weight               32073 non-null  float64
 14  BMI                  32073 non-null  float64
 15  BodyFat              32073 non-null 

In [11]:
# Change datatypes of date columns from object to date
columns_to_change = ['Birthday', 'UIDCreatedDate', 'Weights_CreatedDate',
                     'Weights_UpdatedDate', 'Treatment_StartDate']

merged_df[columns_to_change] = merged_df[columns_to_change].apply(pd.to_datetime)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32073 entries, 0 to 32072
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UID                  32073 non-null  object        
 1   Name                 32073 non-null  object        
 2   LastName             32073 non-null  object        
 3   Gender               32073 non-null  object        
 4   Unit                 32073 non-null  int64         
 5   Birthday             32073 non-null  datetime64[ns]
 6   Age                  32073 non-null  int64         
 7   Height               32073 non-null  int64         
 8   UIDCreatedDate       32073 non-null  datetime64[ns]
 9   User_IsActive        32073 non-null  bool          
 10  ClinicID             32073 non-null  int64         
 11  loginId              0 non-null      float64       
 12  success              32073 non-null  bool          
 13  Weight               32073 non-

In [13]:
# Sort by 'UID', 'UIDCreatedDate', 'TreatmentTypeID', 'Treatment_StartDate',
# 'Weights_CreatedDate', 'Weights_UpdatedDate'
t1 = time.time()

merged_df = merged_df.sort_values(by=['UID',
                                      'UIDCreatedDate',
                                      'TreatmentTypeID',
                                      'Treatment_StartDate',
                                      'Weights_CreatedDate',
                                      'Weights_UpdatedDate'])


t2 = time.time()
print('Time taken to sort dataset ', t2-t1)
merged_df.head()

Time taken to sort dataset  0.021595001220703125


,UID,Name,LastName,Gender,Unit,Birthday,Age,Height,UIDCreatedDate,User_IsActive,...,Bone,VisceralFat,BMR,MuscleMass,Weights_CreatedDate,Weights_UpdatedDate,Weights_IsActive,Weights_IsDelete,TreatmentTypeID,Treatment_StartDate
10192,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2.9,9.7,1502.0,47.4,2023-04-18 18:54:42,2023-04-18 18:54:42,True,False,1,2023-04-16 18:54:42
10189,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,3.0,9.3,1479.2,46.4,2023-04-26 18:54:42,2023-04-26 18:54:42,True,False,1,2023-04-16 18:54:42
10196,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,3.0,9.3,1486.9,46.3,2023-05-01 18:54:42,2023-05-01 18:54:42,True,False,1,2023-04-16 18:54:42
10199,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,3.0,9.3,1476.0,46.0,2023-05-05 18:54:42,2023-05-05 18:54:42,True,False,1,2023-04-16 18:54:42
10203,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,3.1,9.3,1478.2,45.5,2023-05-12 18:54:42,2023-05-12 18:54:42,True,False,1,2023-04-16 18:54:42


## Calculations

In [14]:
# The difference between the treatment start date and the weights created/updated 
# can be used to calculate the month/week from the start of the treatment

# Starting from week/month 0

# Calculate the number of months since treatment start date
merged_df['month'] = (merged_df['Weights_CreatedDate'] - merged_df['Treatment_StartDate']).dt.days / 30.417
merged_df['month'] = merged_df['month'].apply(math.floor)

# Calculate the number of weeks since treatment start date
merged_df['week'] = (merged_df['Weights_CreatedDate'] - merged_df['Treatment_StartDate']).dt.days / 7
merged_df['week'] = merged_df['week'].apply(math.floor)

### All column calculations based on `weekly` cohort

In [18]:
# weigh-in rate - number of times a user has weighed themselves during each week/month of treatment

# Why treatment start date? - Ex- The user might have been in Trt 1 first, switched to Trt 2 and back to 1

merged_df['weigh_in_rate'] = merged_df.groupby(['UID', 
                                                'TreatmentTypeID',
                                                'Treatment_StartDate',
                                                'week'])['Weights_UpdatedDate'].transform('count')

In [19]:
# Patient starting weight
merged_df['patient_starting_weight'] = merged_df.groupby(['UID'])['Weight'].transform('first')

In [20]:
# Treatment starting weight
merged_df['treatment_starting_weight'] = merged_df.groupby(['UID',
                                                            'TreatmentTypeID',
                                                            'Treatment_StartDate'])['Weight'].transform('first')
merged_df.head()

,UID,Name,LastName,Gender,Unit,Birthday,Age,Height,UIDCreatedDate,User_IsActive,...,Weights_UpdatedDate,Weights_IsActive,Weights_IsDelete,TreatmentTypeID,Treatment_StartDate,month,week,weigh_in_rate,patient_starting_weight,treatment_starting_weight
10192,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2023-04-18 18:54:42,True,False,1,2023-04-16 18:54:42,0,0,1,87.4,87.4
10189,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2023-04-26 18:54:42,True,False,1,2023-04-16 18:54:42,0,1,1,87.4,87.4
10196,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2023-05-01 18:54:42,True,False,1,2023-04-16 18:54:42,0,2,2,87.4,87.4
10199,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2023-05-05 18:54:42,True,False,1,2023-04-16 18:54:42,0,2,2,87.4,87.4
10203,004af604-1a06-11ee-a492-eaf1cf2ff621,Cindy,Hartman,Female,1,1964-05-29 04:25:43,59,162,2023-04-16 18:54:42,True,...,2023-05-12 18:54:42,True,False,1,2023-04-16 18:54:42,0,3,2,87.4,87.4


In [21]:
# Treatment total body weight loss

# The differene between the patient weight at the start of the treatment
# and the patient weight at the end of the treatment

# Treatment ending weight
tew = merged_df.groupby(['UID',
                         'TreatmentTypeID',
                         'Treatment_StartDate'])['Weight'].transform('last')

merged_df['treatment_TBWL'] = tew - merged_df['treatment_starting_weight']

In [22]:
# Patient total body weight loss

# The difference between patient weight at the start of the cohort (week/month)
# and patient weight at the start of the next cohort (week/month)

merged_df['wgt_diff'] = merged_df.groupby(['UID',
                                           'TreatmentTypeID',
                                           'Treatment_StartDate',
                                           'week'])['Weight'].transform('first').diff(-1)

# Replace with same cohort_wise weight loss for the respective user cohorts
merged_df['patient_TBWL'] = merged_df.groupby(['UID',
                                               'TreatmentTypeID',
                                               'Treatment_StartDate',
                                               'week'])['wgt_diff'].transform('max')

In [23]:
def filters(df, gender='all', min_age=18, max_age=72, ClinicID=5066):
    """
    :params df(DataFrame): Pandas dataframe to filter
    :params gender (str): Gender of the user; 'Male' or 'Female' or 'all'
    :params min_age (int): Minimum age of the user to be queried
    :params max_age (int): Maximum age of the user to be queried
    :params ClinicID (int): ClinicID of the clinic that the users go to
    
    Filters the data (df) based on Gender, Age and Clinic
    """
    
    # Gender filter
    if gender == 'all':
        df = df
    elif gender == 'Male':
        df = df[df.Gender == 'Male']
    elif gender == 'Female':
        df = df[df.Gender == 'Female']
        
    # Age filter
    # Set min_age and max_age same if you want users of a specific age
    # Change min_age and max_age to get users between a certain age
    df = df[(df.Age >= min_age) & (df.Age <= max_age)]
    
    # Clinic filter
    df = df[df.ClinicID == ClinicID]
    
    return df

def data_pipeline(path_to_data, cohort='week', gender='all',
                  min_age=18, max_age=72, ClinicID=5066):
    """
    :params path_to_data (str): Path to data
    :params cohort (str): To get info monthly, weekly and based on clinic
    :params gender (str): Gender of the user; 'Male' or 'Female' or 'all'
    :params min_age (int): Minimum age of the user to be queried
    :params max_age (int): Maximum age of the user to be queried
    :params ClinicID (int): ClinicID of the clinic that the users go to
    
    Loads dataset, merges them, cleans them, calculates required metrics,
    and returns a dataframe with the 'weigh_in_rate', 'patient_starting_weight',
    'treatment_starting_weight', 'treatment_TBWL', 'patient_TBWL' metrics for each user cohort wise
    """
    
    # Load datasets
    users = pd.read_csv(f'{path_to_data}/users.csv')
    weights = pd.read_csv(f'{path_to_data}/weights.csv')
    treatments = pd.read_csv(f'{path_to_data}/treatments.csv')
    
    
    # Left join dataframes: Users <> Weights, Treatments
    uw = pd.merge(users, weights,
                  left_on='UID', right_on='MasterUserID',
                  how='left')
    merged_df = pd.merge(uw, treatments,
                         left_on='UID', right_on='MasterUserID',
                         how='left')
    
    
    # Drop 2 duplicate UserID columns as the values are same in the 2 columns and UID column
    merged_df = merged_df.drop(columns=['MasterUserID_x', 'MasterUserID_y'])
    
    
    # Rename columns for better understanding
    columns_to_rename = {'CreatedDate_x': 'UIDCreatedDate',
                         'IsActive_x': 'User_IsActive',
                         'CreatedDate_y': 'Weights_CreatedDate',
                         'UpdatedDate': 'Weights_UpdatedDate',
                         'IsActive_y': 'Weights_IsActive',
                         'IsDelete': 'Weights_IsDelete',
                         'StartDate': 'Treatment_StartDate',
                         }
    merged_df = merged_df.rename(columns=columns_to_rename)
    
    
    # Change datatypes of date columns from object to date
    columns_to_change = ['Birthday', 'UIDCreatedDate', 'Weights_CreatedDate',
                         'Weights_UpdatedDate', 'Treatment_StartDate']

    merged_df[columns_to_change] = merged_df[columns_to_change].apply(pd.to_datetime)
    
    
    # Sort the data by 'UID', 'UIDCreatedDate', 'TreatmentTypeID', 'Treatment_StartDate',
    # 'Weights_CreatedDate', 'Weights_UpdatedDate'
    merged_df = merged_df.sort_values(by=['UID',
                                          'UIDCreatedDate',
                                          'TreatmentTypeID',
                                          'Treatment_StartDate',
                                          'Weights_CreatedDate',
                                          'Weights_UpdatedDate'])
    
    
    # The difference between the treatment start date and the weights created/updated 
    # can be used to calculate the month/week from the start of the treatment
    # Starting from week/month 1 i.e., first week of treatment instead of zeroth week

    # Calculate the number of months since treatment start date
    merged_df['month'] = (merged_df['Weights_CreatedDate'] - merged_df['Treatment_StartDate']) / pd.Timedelta(days=30.44)
    merged_df['month'] = merged_df['month'].apply(math.floor) + 1

    # Calculate the number of weeks since treatment start date
    merged_df['week'] = (merged_df['Weights_CreatedDate'] - merged_df['Treatment_StartDate']).dt.days // 7 + 1
    
    
    # weigh-in rate - number of times a user has weighed themselves during each week/month of treatment

    # Why treatment start date? - Ex- The user might have been in Trt 1 first, switched to Trt 2 and back to 1

    merged_df['weigh_in_rate'] = merged_df.groupby(['UID', 
                                                    'TreatmentTypeID',
                                                    'Treatment_StartDate',
                                                    cohort])['Weights_UpdatedDate'].transform('count')
    
    
    # Patient starting weight
    merged_df['patient_starting_weight'] = merged_df.groupby(['UID'])['Weight'].transform('first')
    
    
    # Treatment starting weight
    merged_df['treatment_starting_weight'] = merged_df.groupby(['UID',
                                                                'TreatmentTypeID',
                                                                'Treatment_StartDate'])['Weight'].transform('first')
    
    
    # Treatment total body weight loss

    # The differene between the patient weight at the start of the treatment
    # and the patient weight at the end of the treatment

    # Treatment ending weight
    tew = merged_df.groupby(['UID',
                             'TreatmentTypeID',
                             'Treatment_StartDate'])['Weight'].transform('last')

    merged_df['treatment_TBWL'] = tew - merged_df['treatment_starting_weight']
    
    
    # Patient total body weight loss

    # The difference between patient weight at the start of the cohort (week/month)
    # and patient weight at the start of the next cohort (week/month)

    merged_df['wgt_diff'] = merged_df.groupby(['UID',
                                               'TreatmentTypeID',
                                               'Treatment_StartDate',
                                               cohort])['Weight'].transform('first').diff(-1)

    # Replace with same cohort_wise weight loss for the respective user cohorts
    merged_df['patient_TBWL'] = merged_df.groupby(['UID',
                                                   'TreatmentTypeID',
                                                   'Treatment_StartDate',
                                                   cohort])['wgt_diff'].transform('max')
    # Drop wgt_diff column
    merged_df = merged_df.drop(columns=['wgt_diff'])
    
    
    # Filter
    merged_df = filters(merged_df, gender, min_age, max_age, ClinicID)
    
    
    return merged_df.drop_duplicates()

In [24]:
data_pipeline('Data/', gender='Male', max_age=18)[['Weight', 'week', 'weigh_in_rate',
                                                                    'patient_starting_weight',
                                                                    'treatment_starting_weight',
                                                                    'treatment_TBWL',
                                                                    'patient_TBWL']]

,Weight,week,weigh_in_rate,patient_starting_weight,treatment_starting_weight,treatment_TBWL,patient_TBWL
15914,83.5,1,2,83.5,83.5,-3.0,1.7
15916,83.3,1,2,83.5,83.5,-3.0,1.7
15925,81.8,3,2,83.5,83.5,-3.0,0.3
15922,81.6,3,2,83.5,83.5,-3.0,0.3
15917,81.5,4,4,83.5,83.5,-3.0,0.5
15920,81.4,4,4,83.5,83.5,-3.0,0.5
15923,81.3,4,4,83.5,83.5,-3.0,0.5
15918,81.1,4,4,83.5,83.5,-3.0,0.5
15919,81.0,5,4,83.5,83.5,-3.0,0.1
15926,81.1,5,4,83.5,83.5,-3.0,0.1
